1. Используя язык программирования Python 3 необходимо выполнить предобработку текстовых данных:
- привести все слова к нижнему регистру;
- удалить пунктуацию;
- удалить нерелевантные слова (ссылки, слова на английском, и. т .д);
- удалить стоп слова с помощью готовых словарей ( при необходимости дополнить словарь);
- выполнить лемматизацию ( привести все слова в их начальную словоформу).

2. Отсортировать слова по частоте их употребления и выделить топ 100 слов.

In [1]:
#connection to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#файл с данными не читается встроенным парсером csv из-за смешивания кавычек и запятых, преобразуем файл для удобного чтения в dataframe
import re
data = ''
f = open('/content/drive/My Drive/Colab Notebooks/data.csv', 'r', encoding = 'utf8')
for number, line in enumerate(f):
    # удаляем кавычки вначале строк, удаляем символ точки с запятой, заменяем первые 5 запятых на новый разделитель, который не встречается в файле: `
    line = re.sub('^[\"](.*)', '\\1', line)
    line = line.replace(';','')
    line = re.sub('(.*?),(.*?),(.*?),(.*?),(.*?),(.*?)\n', '\\1`\\2`\\3`\\4`\\5`\\6\n', line)
    data += line
f.close()

In [3]:
import pandas as pd
from io import StringIO
df = pd.read_csv(StringIO(data), sep = '`')
df

,url,author,type,parent_post_url,tag,body
0,https://vk.com/wall505680146_2821,id505680146,Социальные сети,https://vk.com/wall-165936830_65882,0,#show_must_go_on@wazzupbighit#bts#Jin#Suga#RM#...
1,https://vk.com/wall-92124895_6308?reply=6329,id149164749,Социальные сети,https://vk.com/wall-92124895_6308,0,А может кто из Майкопа есть <fm>доброволец</fm...
2,https://vk.com/wall-7138849_2300?reply=2301,lyubars,Социальные сети,https://vk.com/wall-7138849_2300,0,сказать сегодня трём людям правдивый комплимен...
3,https://vk.com/wall-146345852_579,afishadeti51,Социальные сети,https://vk.com/wall-114278140_9723,1,Друзья! Репост<br>Большое социальное мероприят...
4,https://vk.com/wall509637155_124,id509637155,Социальные сети,https://vk.com/wall-137311321_2416,1,ФИЛЬМ МИТРОПОЛИТА ИЛАРИОНА «СРЕТЕНИЕ»<br>Фильм...
...,...,...,...,...,...,...
996,https://vk.com/wall-158214192_23505,stolzvonnordennews,Социальные сети,NaN,1,Новозеландский герой мстил за 11-летнюю шведск...
997,https://vk.com/wall-160027584_3402,kallisti,Социальные сети,NaN,1,УЖЕ ЗАВТРА!!! Будет розыгрыш суперпризов!!!Бл...
998,https://vk.com/wall-122022858_7401,rbctvperm,Социальные сети,NaN,1,РБК-Пермь. Итоги 15.03.19 <br>Информационные и...
999,https://vk.com/wall-167593733_1184,club167593733,Социальные сети,NaN,1,Ирина Набатова <br>Администратор · 13 мин. <br...


In [4]:
#удаляем тэги html
df['body'] = df['body'].str.replace(r'\<[^\>]*\>', ' ')
#удаляем латинские символы, числа, табуляцию, специальный пробельный символ и знак подчеркивания (ссылки в результате исполнения данной команды тоже будут удалены)
df['body'] = df['body'].str.replace(r'[A-Za-z\d\t\xa0_]', ' ')
#удаляем знаки препинания (без знака подчеркивания)
df['body'] = df['body'].str.replace(r'[^А-аЯ-яЁё\s]', ' ')
#приводим к нижнему регистру
df['body'] = df['body'].str.lower()
#удаляем лишние пробелы
df['body'] = df['body'].str.replace(r' +', ' ')

In [5]:
#скачваем модуль "stopwords" библиотеки nltk
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Package stopwords is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [6]:
#определяем функцию удаления стоп-слов
from nltk.corpus import stopwords
def clean_stopwords(line):
  stopWords = stopwords.words("russian")
  stopWords2 = stopwords.words("english")
  return [word for word in line if word not in stopWords and word not in stopWords2]

In [7]:
df['body'] = df['body'].str.split()
df['body'] = df['body'].apply(clean_stopwords)
df

,url,author,type,parent_post_url,tag,body
0,https://vk.com/wall505680146_2821,id505680146,Социальные сети,https://vk.com/wall-165936830_65882,0,"[пост, пополняться, хосок, отсчет, показать, п..."
1,https://vk.com/wall-92124895_6308?reply=6329,id149164749,Социальные сети,https://vk.com/wall-92124895_6308,0,"[майкопа, доброволец, коррекцию]"
2,https://vk.com/wall-7138849_2300?reply=2301,lyubars,Социальные сети,https://vk.com/wall-7138849_2300,0,"[сказать, сегодня, трём, людям, правдивый, ком..."
3,https://vk.com/wall-146345852_579,afishadeti51,Социальные сети,https://vk.com/wall-114278140_9723,1,"[друзья, репост, большое, социальное, мероприя..."
4,https://vk.com/wall509637155_124,id509637155,Социальные сети,https://vk.com/wall-137311321_2416,1,"[фильм, митрополита, илариона, сретение, фильм..."
...,...,...,...,...,...,...
996,https://vk.com/wall-158214192_23505,stolzvonnordennews,Социальные сети,NaN,1,"[новозеландский, герой, мстил, летнюю, шведску..."
997,https://vk.com/wall-160027584_3402,kallisti,Социальные сети,NaN,1,"[завтра, розыгрыш, суперпризов, благотворитель..."
998,https://vk.com/wall-122022858_7401,rbctvperm,Социальные сети,NaN,1,"[рбк, пермь, итоги, информационные, итоги, нед..."
999,https://vk.com/wall-167593733_1184,club167593733,Социальные сети,NaN,1,"[ирина, набатова, администратор, мин, праздник..."


In [8]:
#используем pymorphy2 для лемматизации
!pip install pymorphy2

In [9]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def lemmatize(line):
    return [morph.parse(word)[0].normal_form for word in line]

In [10]:
df['body'] = df['body'].apply(lemmatize)
df

,url,author,type,parent_post_url,tag,body
0,https://vk.com/wall505680146_2821,id505680146,Социальные сети,https://vk.com/wall-165936830_65882,0,"[пост, пополняться, хосока, отсчёт, показать, ..."
1,https://vk.com/wall-92124895_6308?reply=6329,id149164749,Социальные сети,https://vk.com/wall-92124895_6308,0,"[майкоп, доброволец, коррекция]"
2,https://vk.com/wall-7138849_2300?reply=2301,lyubars,Социальные сети,https://vk.com/wall-7138849_2300,0,"[сказать, сегодня, три, человек, правдивый, ко..."
3,https://vk.com/wall-146345852_579,afishadeti51,Социальные сети,https://vk.com/wall-114278140_9723,1,"[друг, репост, большой, социальный, мероприяти..."
4,https://vk.com/wall509637155_124,id509637155,Социальные сети,https://vk.com/wall-137311321_2416,1,"[фильм, митрополит, иларион, сретение, фильм, ..."
...,...,...,...,...,...,...
996,https://vk.com/wall-158214192_23505,stolzvonnordennews,Социальные сети,NaN,1,"[новозеландский, герой, мстить, летний, шведск..."
997,https://vk.com/wall-160027584_3402,kallisti,Социальные сети,NaN,1,"[завтра, розыгрыш, суперприз, благотворительны..."
998,https://vk.com/wall-122022858_7401,rbctvperm,Социальные сети,NaN,1,"[рбк, пермь, итог, информационный, итог, недел..."
999,https://vk.com/wall-167593733_1184,club167593733,Социальные сети,NaN,1,"[ирина, набатов, администратор, мина, праздник..."


In [11]:
#Соединим все посты в один массив и посчитаем частоту слов
from collections import Counter
words_freq = Counter(df['body'].sum())

In [12]:
words_freq.most_common(100)

[('это', 1499),
 ('который', 1368),
 ('год', 1323),
 ('свой', 1238),
 ('человек', 1237),
 ('день', 1043),
 ('благотворительный', 844),
 ('ребёнок', 740),
 ('помощь', 696),
 ('наш', 692),
 ('жизнь', 657),
 ('время', 622),
 ('быть', 577),
 ('показать', 525),
 ('полностью', 511),
 ('весь', 501),
 ('другой', 498),
 ('стать', 490),
 ('деньга', 487),
 ('один', 479),
 ('дело', 462),
 ('ваш', 445),
 ('добрый', 439),
 ('каждый', 438),
 ('мочь', 436),
 ('очень', 419),
 ('друг', 418),
 ('новый', 415),
 ('такой', 412),
 ('россия', 404),
 ('фонд', 403),
 ('помочь', 390),
 ('дом', 384),
 ('сам', 373),
 ('должный', 360),
 ('тот', 360),
 ('первый', 357),
 ('мир', 355),
 ('работа', 350),
 ('доброта', 346),
 ('г', 344),
 ('благотворительность', 340),
 ('делать', 339),
 ('самый', 339),
 ('проект', 337),
 ('семья', 334),
 ('организация', 333),
 ('мы', 322),
 ('просто', 320),
 ('сделать', 302),
 ('также', 295),
 ('нужно', 295),
 ('иметь', 292),
 ('мой', 292),
 ('говорить', 285),
 ('слово', 285),
 ('хотеть'

In [13]:
#результаты до сих пор содержат некоторые стопслова, дополним фильтрацию собственным списком стопслов
def clean_more_stopwords(line):
  with open('/content/drive/My Drive/Colab Notebooks/stopwords.txt') as file:
    stopWords3 = [row.strip() for row in file]
  return [word for word in line if word not in stopWords3]
  
df['body'] = df['body'].apply(clean_more_stopwords)

In [14]:
#финальный результат
words_freq = Counter(df['body'].sum())
words_freq.most_common(100)

[('благотворительный', 844),
 ('ребёнок', 740),
 ('помощь', 696),
 ('показать', 525),
 ('деньга', 487),
 ('добрый', 439),
 ('фонд', 403),
 ('помочь', 390),
 ('доброта', 346),
 ('благотворительность', 340),
 ('проект', 337),
 ('семья', 334),
 ('организация', 333),
 ('праздник', 264),
 ('участие', 255),
 ('группа', 250),
 ('февраль', 241),
 ('сбор', 237),
 ('помогать', 235),
 ('рубль', 230),
 ('любой', 225),
 ('средство', 217),
 ('сумма', 216),
 ('мероприятие', 213),
 ('пара', 211),
 ('карта', 206),
 ('счёт', 203),
 ('возможность', 202),
 ('март', 200),
 ('поддержка', 189),
 ('принять', 187),
 ('иоанн', 186),
 ('пройти', 183),
 ('являться', 183),
 ('история', 182),
 ('больший', 181),
 ('добро', 176),
 ('любовь', 172),
 ('вещь', 172),
 ('огромный', 172),
 ('проявление', 172),
 ('цель', 171),
 ('право', 171),
 ('центр', 170),
 ('известный', 169),
 ('проблема', 168),
 ('заниматься', 165),
 ('область', 160),
 ('международный', 158),
 ('родитель', 157),
 ('настоящий', 157),
 ('начало', 157),
